# Обработка данных для оценки токсичности текста

In [ ]:
!unzip /content/drive/MyDrive/thesis_datasets/dataset.txt.zip

Archive:  /content/drive/MyDrive/thesis_datasets/dataset.txt.zip
  inflating: dataset.txt             


In [ ]:
!unzip /content/drive/MyDrive/thesis_datasets/labeled.csv.zip

Archive:  /content/drive/MyDrive/thesis_datasets/labeled.csv.zip
  inflating: labeled.csv             


In [1]:
import re
import pandas as pd

In [2]:
with open("/content/drive/MyDrive/thesis_datasets/dataset.txt", "r") as f:
  dataset = f.readlines()

In [3]:
dataset[:10]

['__label__INSULT скотина! что сказать\n',
 '__label__NORMAL я сегодня проезжала по рабочей и между домами снитенко и гомолысовой магазином ( на пустыре) бежала кошка похожего окраса. может, я и ошиблась, но необычный окрас бросился в глаза.\n',
 '__label__NORMAL очередной лохотрон. зачем придумывать очередной налог на воздух, если можно обьявить инсульт и грипп- пандемией! и лихо на придурках зарабатывать годами на штрафах, фейковых вакцинах, всевозможных платных тестах, продажей масок и перчаток по баснословным ценам.. самое смешное, что бараны блеют и верят пастуху, телевизору. живут как под гипнозом. не думая, не глядя по сторонам.\n',
 '__label__NORMAL ретро дежавю ... сложно понять чужое сердце , лиш ощутить музыкой видимо\n',
 '__label__NORMAL а когда мы статус агрогородка получили?\n',
 '__label__NORMAL 2 августа поздно вечером нашли вот такую потеряшку в районе высоток на победе. девочка явно домашняя, в новом ошейнике. обращаться +7 989 816-43-42\n',
 '__label__NORMAL вчера н

In [4]:
len(dataset)

248290

In [7]:
%%time
labels = []
for line in dataset:
  labels_in_line = re.findall("(?<=__label__)\w+", line)
  for label in labels_in_line:
    labels.append(label)
unique_labels = set(labels)

CPU times: user 803 ms, sys: 19.9 ms, total: 823 ms
Wall time: 826 ms


In [8]:
unique_labels

{'INSULT', 'NORMAL', 'OBSCENITY', 'THREAT'}

In [15]:
%%time
texts_and_labels = {}
for line in dataset:
  labels_in_line = re.findall("(?<=__label__)\w+", line)
  text = re.sub("__label__(?<=__label__)\w+", '', line)
  texts_and_labels[text] = labels_in_line

CPU times: user 1.44 s, sys: 89 ms, total: 1.53 s
Wall time: 1.53 s


In [17]:
texts = texts_and_labels.keys()
labels = texts_and_labels.values()

In [18]:
labeled_dataset = pd.DataFrame(texts, columns=['texts'])

In [19]:
labeled_dataset['insult'] = [1 if 'INSULT' in label else 0 for label in labels]
labeled_dataset['obscenity'] = [1 if 'OBSCENITY' in label else 0 for label in labels]
labeled_dataset['threat'] = [1 if 'THREAT' in label else 0 for label in labels]
labeled_dataset['normal'] = [1 if 'NORMAL' in label else 0 for label in labels]

In [20]:
labeled_dataset.head(30)

,texts,insult,obscenity,threat,normal
0,скотина! что сказать\n,1,0,0,0
1,я сегодня проезжала по рабочей и между домами...,0,0,0,1
2,очередной лохотрон. зачем придумывать очередн...,0,0,0,1
3,"ретро дежавю ... сложно понять чужое сердце ,...",0,0,0,1
4,а когда мы статус агрогородка получили?\n,0,0,0,1
5,2 августа поздно вечером нашли вот такую поте...,0,0,0,1
6,вчера надыбала новые стикеры #u2a94ec7fabs#\n,0,0,0,1
7,", заколоть этого плешивого урода что бы крякну...",1,0,1,0
8,а еще на стоянке никто не проверяет безопасно...,0,0,0,1
9,"красота..!! если есть, что показать??!! почем...",0,0,0,1


In [21]:
labeled_dataset.to_csv("/content/drive/MyDrive/thesis_datasets/labeled_big_dataset")

In [2]:
big_dataset = pd.read_csv("/content/drive/MyDrive/thesis_datasets/labeled_big_dataset")

In [4]:
big_dataset.drop(labels=['Unnamed: 0'], axis=1, inplace=True)

In [5]:
big_dataset.head()

,texts,insult,obscenity,threat,normal
0,скотина! что сказать\n,1,0,0,0
1,я сегодня проезжала по рабочей и между домами...,0,0,0,1
2,очередной лохотрон. зачем придумывать очередн...,0,0,0,1
3,"ретро дежавю ... сложно понять чужое сердце ,...",0,0,0,1
4,а когда мы статус агрогородка получили?\n,0,0,0,1


In [7]:
big_dataset['toxic'] = [1 if big_dataset['normal'][i] == 0 else 0 for i in range(len(big_dataset))]

In [8]:
big_dataset.head()

,texts,insult,obscenity,threat,normal,toxic
0,скотина! что сказать\n,1,0,0,0,1
1,я сегодня проезжала по рабочей и между домами...,0,0,0,1,0
2,очередной лохотрон. зачем придумывать очередн...,0,0,0,1,0
3,"ретро дежавю ... сложно понять чужое сердце ,...",0,0,0,1,0
4,а когда мы статус агрогородка получили?\n,0,0,0,1,0


In [9]:
small_dataset = pd.read_csv("/content/drive/MyDrive/thesis_datasets/labeled.csv")

In [10]:
small_dataset.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [11]:
len(small_dataset)

14412

In [12]:
small_dataset = small_dataset.rename({'comment': 'texts'}, axis=1)

In [13]:
small_dataset.head()

,texts,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [16]:
dataset = pd.concat([big_dataset, small_dataset], axis=0)

In [17]:
dataset.head()

,texts,insult,obscenity,threat,normal,toxic
0,скотина! что сказать\n,1.0,0.0,0.0,0.0,1.0
1,я сегодня проезжала по рабочей и между домами...,0.0,0.0,0.0,1.0,0.0
2,очередной лохотрон. зачем придумывать очередн...,0.0,0.0,0.0,1.0,0.0
3,"ретро дежавю ... сложно понять чужое сердце ,...",0.0,0.0,0.0,1.0,0.0
4,а когда мы статус агрогородка получили?\n,0.0,0.0,0.0,1.0,0.0


In [18]:
dataset

,texts,insult,obscenity,threat,normal,toxic
0,скотина! что сказать\n,1.0,0.0,0.0,0.0,1.0
1,я сегодня проезжала по рабочей и между домами...,0.0,0.0,0.0,1.0,0.0
2,очередной лохотрон. зачем придумывать очередн...,0.0,0.0,0.0,1.0,0.0
3,"ретро дежавю ... сложно понять чужое сердце ,...",0.0,0.0,0.0,1.0,0.0
4,а когда мы статус агрогородка получили?\n,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,NaN,NaN,NaN,NaN,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,NaN,NaN,NaN,NaN,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",NaN,NaN,NaN,NaN,0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,NaN,NaN,NaN,NaN,1.0


In [19]:
len(dataset)

262702

In [22]:
dataset.to_csv("/content/drive/MyDrive/thesis_datasets/toxic_dataset.csv", index=False)